## Resilient Distributed Datasets (RDDs) - Lab

Resilient Distributed Datasets (RDD) are fundamental data structures of Spark. An RDD is essentially the Spark representation of a set of data, spread across multiple machines, with APIs to let you act on it. An RDD can come from any data source, e.g. text files, a database, a JSON file, etc.


## Objectives

You will be able to:

- Apply the map(func) transformation to a given function on all elements of an RDD in different partitions 
- Apply a map transformation for all elements of an RDD 
- Compare the difference between a transformation and an action within RDDs 
- Use collect(), count(), and take() actions to trigger spark transformations  
- Use filter to select data that meets certain specifications within an RDD 
- Set number of partitions for parallelizing RDDs 
- Create RDDs from Python collections 


## What are RDDs? 

To get a better understanding of RDDs, let's break down each one of the components of the acronym RDD:

Resilient: RDDs are considered "resilient" because they have built-in fault tolerance. This means that even if one of the nodes goes offline, RDDs will be able to restore the data. This is already a huge advantage compared to standard storage. If a standard computer dies while performing an operation, all of its memory will be lost in the process. With RDDs, multiple nodes can go offline, and the action will still be held in working memory.

Distributed: The data is contained on multiple nodes of a cluster-computing operation. It is efficiently partitioned to allow for parallelism.

Dataset: The dataset has been * partitioned * across the multiple nodes. 

RDDs are the building block upon which more high-level Spark operations are based upon. Chances are, if you are performing an action using Spark, the operation involves RDDs. 



Key Characteristics of RDDs:

- Immutable: Once an RDD is created, it cannot be modified. 
- Lazily Evaluated: RDDs will not be evaluated until an action is triggered. Essentially, when RDDs are created, they are programmed to perform some action, but that function will not get activated until it is explicitly called. The reason for lazy evaluation is that allows users to organize the actions of their Spark program into smaller actions. It also saves unnecessary computation and memory load.
- In-Memory: The operations in Spark are performed in-memory rather than in the database. This is what allows Spark to perform fast operations with very large quantities of data.




### RDD Transformations vs Actions

In Spark, we first create a __base RDD__ and then apply one or more transformations to that base RDD following our processing needs. Being immutable means, **once an RDD is created, it cannot be changed**. As a result, **each transformation of an RDD creates a new RDD**. Finally, we can apply one or more **actions** to the RDDs. Spark uses lazy evaluation, so transformations are not actually executed until an action occurs.


<img src="./images/rdd_diagram.png" width=500>

### Transformations

Transformations create a new dataset from an existing one by passing each dataset element through a function and returning a new RDD representing the results. In short, creating an RDD from an existing RDD is ‘transformation’.
All transformations in Spark are lazy. They do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result that needs to be returned to the driver program.
A transformation is an RDD that returns another RDD, like map, flatMap, filter, reduceByKey, join, cogroup, etc.

### Actions
Actions return final results of RDD computations. Actions trigger execution using lineage graph to load the data into original RDD and carry out all intermediate transformations and return the final results to the driver program or writes it out to the file system. An action returns a value (to a Spark driver - the user program).

Here are some key transformations and actions that we will explore.


| Transformations   | Actions       |
|-------------------|---------------|
| map(func)         | reduce(func)  |
| filter(func)      | collect()     |
| groupByKey()      | count()       |
| reduceByKey(func) | first()       |
| mapValues(func)   | take()        |
| sample()          | countByKey()  |
| distinct()        | foreach(func) |
| sortByKey()       |               |


Let's see how transformations and actions work through a simple example. In this example, we will perform several actions and transformations on RDDs in order to obtain a better understanding of Spark processing. 

### Create a Python collection 

We need some data to start experimenting with RDDs. Let's create some sample data and see how RDDs handle it. To practice working with RDDs, we're going to use a simple Python list.

- Create a Python list `data` of integers between 1 and 1000 using the `range()` function. 
- Sanity check: confirm the length of the list (it should be 1000)

In [1]:
nums = list(range(1,1001))
len(nums)

1000

### Initialize an RDD

When using Spark to make computations, datasets are treated as lists of entries. Those lists are split into different partitions across different cores or different computers. Each list of data held in memory is a partition of the RDD. The reason why Spark is able to make computations far faster than other big data processing languages is that it allows all data to be stored __in-memory__, which allows for easy access to the data and, in turn, high-speed processing. Here is an example of how the alphabet might be split into different RDDs and held across a distributed collection of nodes:

<img src ="./images/partitions_1.png" width ="500">  
To initialize an RDD, first import `pyspark` and then create a SparkContext assigned to the variable `sc`. Use `'local[*]'` as the master.

In [2]:
import pyspark
sc = pyspark.SparkContext('local[*]')

Once you've created the SparkContext, you can use the `.parallelize()` method to create an RDD that will distribute the list of numbers across multiple cores. Here, create one called `rdd` with 10 partitions using `data` as the collection you are parallelizing.

In [3]:
rdd = sc.parallelize(nums, numSlices=10)
print(type(rdd))

<class 'pyspark.rdd.RDD'>


Determine how many partitions are being used with this RDD with the `.getNumPartitions()` method.

In [4]:
rdd.getNumPartitions()

10

### Basic descriptive RDD actions

Let's perform some basic operations on our RDD. In the cell below, use the methods:
* `count`: returns the total count of items in the RDD 
* `first`: returns the first item in the RDD
* `take`: returns the first `n` items in the RDD
* `top`: returns the top `n` items
* `collect`: returns everything from your RDD


It's important to note that in a big data context, calling the collect method will often take a very long time to execute and should be handled with care!

In [5]:
rdd.count()

1000

In [6]:
rdd.first()

1

In [7]:
rdd.take(10)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
rdd.top(10)

[1000, 999, 998, 997, 996, 995, 994, 993, 992, 991]

In [9]:
## Note: When you are dealing with big data, this could make your computer crash! It's best to avoid using the collect() method
rdd.collect()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

## Map functions

Now that you've been working a little bit with RDDs, let's make this a little more interesting. Imagine you're running a hot new e-commerce startup called BuyStuff, and you're trying to track of how much it charges customers from each item sold. In the next cell, we're going to create simulated data by multiplying the values 1-1000 with a random number from 0-1.

In [10]:
import random
import numpy as np

nums = np.array(range(1, 1001))
sales_figures = nums * np.random.rand(1000)
sales_figures

array([5.72204100e-01, 1.62100592e+00, 2.59460599e-04, 2.66588381e+00,
       4.67649904e+00, 3.33367338e+00, 5.14629560e+00, 3.55023821e+00,
       4.08778866e+00, 4.41397478e+00, 7.32074939e+00, 8.57762222e-01,
       8.01005648e-01, 7.27319603e-01, 2.55822044e+00, 1.23563111e+01,
       2.62739129e-01, 2.19899722e+00, 1.15001198e+01, 2.50557521e+00,
       1.53691270e+01, 7.50654558e+00, 3.42602749e+00, 1.21969695e+01,
       9.45640924e+00, 2.49844838e+01, 2.44772684e+01, 8.10193652e+00,
       1.96868406e+00, 8.16154528e+00, 2.42420840e+01, 6.40319921e+00,
       3.71136738e+00, 3.02402304e+01, 1.91404790e+01, 3.09732160e+01,
       3.42726386e+01, 2.65859379e+01, 3.48108110e+01, 3.35405774e+01,
       1.68553943e+01, 1.20446634e+00, 3.33975696e+01, 5.14240534e-02,
       3.19439257e+00, 1.77229712e+01, 4.46606405e+01, 2.89411613e+01,
       1.99931340e+00, 3.41334202e+01, 1.62188346e+01, 2.96551066e+01,
       5.05279522e+01, 1.78386551e+01, 3.41910300e+01, 7.08174128e+00,
      

We now have sales prices for 1000 items currently for sale at BuyStuff. Now create an RDD called `price_items` using the newly created data with 10 slices. After you create it, use one of the basic actions to see what's in the RDD.

In [11]:
price_items = sc.parallelize(sales_figures, numSlices=10)
price_items.take(4)

[0.5722041001271304,
 1.6210059212960308,
 0.0002594605990777765,
 2.665883814323598]

Now let's perform some operations on this simple dataset. To begin with, create a function that will take into account how much money BuyStuff will receive after sales tax has been applied (assume a sales tax of 8%). To make this happen, create a function called `sales_tax()` that returns the amount of money our company will receive after the sales tax has been applied. The function will have this parameter:

* `item`: (float) number to be multiplied by the sales tax.


Apply that function to the rdd by using the `.map()` method and assign it to a variable `revenue_minus_tax`

In [12]:
def sales_tax(num):
    return (num / 1.08)

revenue_minus_tax = price_items.map(sales_tax)

Remember, Spark has __lazy evaluation__, which means that the `sales_tax()` function is a transformer that is not executed until you call an action. Use one of the collection methods to execute the transformer now a part of the RDD and observe the contents of the `revenue_minus_tax` rdd.

In [13]:
revenue_minus_tax.take(10)

[0.5298186112288245,
 1.500931408607436,
 0.00024024129544238565,
 2.4684109391885163,
 4.330091704416674,
 3.086734609383453,
 4.765088522133603,
 3.2872576014085344,
 3.7849895017695188,
 4.087013685734022]

### Lambda Functions

Note that you can also use lambda functions if you want to quickly perform simple operations on data without creating a function. Let's assume that BuyStuff has also decided to offer a 10% discount on all of their items on the pre-tax amounts of each item. Use a lambda function within a `.map()` method to apply the additional 10% loss in revenue for BuyStuff and assign the transformed RDD to a new RDD called `discounted`.

In [14]:
discounted = revenue_minus_tax.map(lambda x : x*0.9)

In [15]:
discounted.take(10)

[0.476836750105942,
 1.3508382677466924,
 0.0002162171658981471,
 2.2215698452696646,
 3.8970825339750066,
 2.778061148445108,
 4.288579669920242,
 2.958531841267681,
 3.406490551592567,
 3.67831231716062]

## Chaining Methods

You are also able to chain methods together with Spark. In one line, remove the tax and discount from the revenue of BuyStuff and use a collection method to see the 15 costliest items.

In [16]:
price_items.map(sales_tax).map(lambda x : x*0.9).top(15)

[785.4968707389128,
 783.2097326417894,
 771.4108560644618,
 760.2047769946646,
 757.8096818400659,
 754.1140672047226,
 747.4579766399983,
 746.6495833269893,
 733.1621402821269,
 729.013798473382,
 719.0298050484045,
 718.1201926552071,
 717.1131866662154,
 716.0356130590471,
 712.2262956026005]

## RDD Lineage


We are able to see the full lineage of all the operations that have been performed on an RDD by using the `RDD.toDebugString()` method. As your transformations become more complex, you are encouraged to call this method to get a better understanding of the dependencies between RDDs. Try calling it on the `discounted` RDD to see what RDDs it is dependent on.

In [17]:
discounted.toDebugString()

b'(10) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |   ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262 []'

### Map vs. Flatmap

Depending on how you want your data to be outputted, you might want to use `.flatMap()` rather than a simple `.map()`. Let's take a look at how it performs operations versus the standard map. Let's say we wanted to maintain the original amount BuyStuff receives for each item as well as the new amount after the tax and discount are applied. Create a map function that will return a tuple with (original price, post-discount price).

In [18]:
mapped = price_items.map(lambda x: (x, (x/1.08)*0.9))
print(mapped.count())
print(mapped.take(10))

1000
[(0.5722041001271304, 0.476836750105942), (1.6210059212960308, 1.3508382677466924), (0.0002594605990777765, 0.0002162171658981471), (2.665883814323598, 2.2215698452696646), (4.6764990407700076, 3.8970825339750066), (3.3336733781341295, 2.778061148445108), (5.146295603904291, 4.288579669920242), (3.5502382095212175, 2.958531841267681), (4.08778866191108, 3.406490551592567), (4.413974780592744, 3.67831231716062)]


Note that we have 1000 tuples created to our specification. Let's take a look at how `.flatMap()` differs in its implementation. Use the `.flatMap()` method with the same function you created above.

In [19]:
flat_mapped = price_items.flatMap(lambda x : (x, (x/1.08)*0.9 ))
print(flat_mapped.count())
print(flat_mapped.take(10))

2000
[0.5722041001271304, 0.476836750105942, 1.6210059212960308, 1.3508382677466924, 0.0002594605990777765, 0.0002162171658981471, 2.665883814323598, 2.2215698452696646, 4.6764990407700076, 3.8970825339750066]


Rather than being represented by tuples, all of the  values are now on the same level. When we are trying to combine different items together, it is sometimes necessary to use `.flatMap()` rather than `.map()` in order to properly reduce to our specifications. This is not one of those instances, but in the upcoming lab, you just might have to use it.

## Filter
After meeting with some external consultants, BuyStuff has determined that its business will be more profitable if it focuses on higher ticket items. Now, use the `.filter()` method to select items that bring in more than $300 after tax and discount have been removed. A filter method is a specialized form of a map function that only returns the items that match a certain criterion. In the cell below:
* use a lambda function within a `.filter()` method to meet the consultant's suggestion's specifications. set `RDD = selected_items`
* calculate the total number of items remaining in BuyStuff's inventory

In [20]:
selected_items = discounted.filter(lambda x: x>300)
selected_items.count()

269

## Reduce

Reduce functions are where you are in some way combing all of the variables that you have mapped out. Here is an example of how a reduce function works when the task is to sum all values:

<img src = "./images/reduce_function.png" width = "600">  


As you can see, the operation is performed within each partition first, after which, the results of the computations in each partition are combined to come up with one final answer.  

Now it's time to figure out how much money BuyStuff would make from selling one of each of its items after they've reduced their inventory. Use the `.reduce()` method with a lambda function to add up all of the values in the RDD. Your lambda function should have two variables. 

In [21]:
selected_items.reduce(lambda x,y: x + y)

126743.10295037908

The time has come for BuyStuff to open up shop and start selling its goods. It only has one of each item, but it's allowing 50 lucky users to buy as many items as they want while they remain in stock. Within seconds, BuyStuff is sold out. Below, you'll find the sales data in an RDD with tuples of (user, item bought).

In [22]:
import random
random.seed(42)
# generating simulated users that have bought each item
sales_data = selected_items.map(lambda x: (random.randint(1, 50), x))

sales_data.take(7)

[(16, 303.6633090522931),
 (12, 301.9747493513825),
 (41, 332.987854404504),
 (25, 313.0425049822443),
 (39, 352.449328173448),
 (13, 325.5482085963404),
 (22, 305.3236976302504)]

It's time to determine some basic statistics about BuyStuff users.

Let's start off by creating an RDD that determines how much each user spent in total.
To do this we can use a method called `.reduceByKey()` to perform reducing operations while grouping by keys. After you have calculated the total, use the `.sortBy()` method on the RDD to rank the users from the highest spending to the least spending. 

In [23]:
total_spent = sales_data.reduceByKey(lambda x, y: x + y)
total_spent.take(10)

[(20, 2541.5612708632902),
 (30, 2842.196479723666),
 (10, 2433.2873228006356),
 (50, 1342.5935254984502),
 (40, 993.6645115059539),
 (31, 2112.533025576309),
 (11, 2075.779056240018),
 (21, 4837.575925242112),
 (41, 2978.8103715337465),
 (1, 1410.3729605117962)]

In [24]:
total_spent.sortBy(lambda x: x[1],ascending = False).collect()

[(16, 5870.1886436860605),
 (5, 5762.231371017182),
 (24, 5307.948308425568),
 (21, 4837.575925242112),
 (6, 4278.166901431887),
 (38, 3998.5263000055465),
 (37, 3968.9769008765015),
 (35, 3861.9215458480153),
 (18, 3537.4028517348856),
 (19, 3373.5093795259554),
 (49, 3355.8931981355727),
 (39, 3287.1824988140866),
 (22, 3130.5740973071906),
 (41, 2978.8103715337465),
 (12, 2914.5227701477043),
 (42, 2871.226336932342),
 (30, 2842.196479723666),
 (43, 2789.0646832047637),
 (13, 2660.3671140984648),
 (20, 2541.5612708632902),
 (33, 2470.4170444809583),
 (48, 2443.9554236804406),
 (10, 2433.2873228006356),
 (44, 2394.8079838412673),
 (45, 2338.506814659404),
 (34, 2298.6839707459903),
 (26, 2274.1128588867473),
 (15, 2137.4356196451313),
 (31, 2112.533025576309),
 (32, 2077.7050337631345),
 (11, 2075.779056240018),
 (47, 2019.9838606609244),
 (2, 1910.4211815846375),
 (29, 1871.859886872262),
 (17, 1844.4754730036975),
 (36, 1777.6164806563074),
 (46, 1694.2563451997732),
 (28, 1659.985

Next, let's determine how many items were bought per user. This can be solved in one line using an RDD method. After you've counted the total number of items bought per person, sort the users from most number of items bought to least number of items. Time to start a customer loyalty program!

In [25]:
total_items = sales_data.countByKey()
sorted(total_items.items(),key=lambda kv:kv[1],reverse=True)

[(50, 12),
 (21, 11),
 (39, 10),
 (48, 9),
 (40, 9),
 (31, 9),
 (24, 8),
 (28, 8),
 (43, 8),
 (26, 7),
 (47, 7),
 (3, 7),
 (41, 7),
 (13, 7),
 (36, 7),
 (19, 7),
 (20, 7),
 (11, 6),
 (29, 6),
 (34, 6),
 (9, 6),
 (25, 6),
 (23, 5),
 (27, 5),
 (35, 5),
 (1, 5),
 (44, 5),
 (14, 5),
 (45, 5),
 (2, 4),
 (8, 4),
 (6, 4),
 (5, 4),
 (15, 4),
 (33, 4),
 (42, 4),
 (10, 4),
 (16, 4),
 (12, 3),
 (46, 3),
 (4, 3),
 (30, 3),
 (32, 3),
 (17, 3),
 (7, 2),
 (49, 2),
 (22, 2),
 (18, 2),
 (38, 1),
 (37, 1)]

### Additional Reading

- [The original paper on RDDs](https://cs.stanford.edu/~matei/papers/2012/nsdi_spark.pdf)
- [RDDs in Apache Spark](https://data-flair.training/blogs/create-rdds-in-apache-spark/)
- [Programming with RDDs](https://runawayhorse001.github.io/LearningApacheSpark/rdd.html)
- [RDD Transformations and Actions Summary](https://www.analyticsvidhya.com/blog/2016/10/using-pyspark-to-perform-transformations-and-actions-on-rdd/)

## Summary

In this lab we went through a brief introduction to RDD creation from a Python collection, setting a number of logical partitions for an RDD and extracting lineage. We also used transformations and actions to perform calculations across RDDs on a distributed setup. In the next lab, you'll get the chance to apply these transformations on different books to calculate word counts and various statistics.
